In [1]:
cd assign_529/

/home/bhawna/assign_529


In [ ]:
import csv
moviemap={}
moviemap_inv={}
with open('./dataset/ratings.csv','rt')as f:
    data = csv.reader(f)
    i=0
    cnt=0
    for row in data:
        if i!=0:
            x=int(row[1])
            if x in moviemap.keys():
                print("present")
            else:
                moviemap[int(row[1])]=cnt
                moviemap_inv[cnt]=int(row[1])
                cnt=cnt+1
        i=i+1
#print(cnt) cnt = 9724
rows=611
cols=9725
rating_mat=[[0 for i in range(cols)] for j in range(rows)]
test_mat=[[0 for i in range(cols)] for j in range(rows)]
with open('./dataset/ratings.csv','rt')as f:
    data = csv.reader(f)
    i=0
    for row in data:
        if i!=0:
            if i%5==0:
                test_mat[int(row[0])][moviemap[int(row[1])]]=float(row[2])
            else:
                rating_mat[int(row[0])][moviemap[int(row[1])]]=float(row[2])
        i=i+1

In [3]:
import numpy as np
import copy

# K-Means Clustering

In [17]:
# def k_means(r,u1,u2,rmat,k=5):
k=8
data = np.array(rating_mat)
c = cols
n=rows
distances = np.zeros((n,k))
mean = np.mean(data, axis = 0)
std = np.std(data, axis = 0)
centers = np.random.randn(k,c)*std + mean
centers_old = np.zeros(centers.shape)
centers_new = copy.deepcopy(centers)
clusters = np.zeros(n)
error = np.linalg.norm(centers_new-centers_old)
while error!=0:
    for i in range(k):
        distances[:,i]=np.linalg.norm(data-centers[i],axis=1)
    clusters=np.argmin(distances,axis=1)
    centers_old = copy.deepcopy(centers_new)
    for i in range(k):
        centers_new[i] = np.mean(data[clusters==i],axis=0)
    error = np.linalg.norm(centers_new-centers_old)

Reading the  similarity matrix already calculated for users from file

In [14]:
sim_u=[[0 for i in range(rows)] for j in range(rows)]#similarity matrix using pearson similarity (user-user)

In [15]:
with open('./sim.csv','rt')as f:
    data = csv.reader(f)
    i=0
    for row in data:
        sim_u[int(row[0])][int(row[1])]=float(row[2])
        sim_u[int(row[1])][int(row[0])]=float(row[2])    

In [16]:
f.close()

Grouping the cells in same cluster together in an array indexed by cluster number

In [18]:
cl_array=[[] for i in range(k)]
for i in range(rows):
    x = clusters[i]
    cl_array[x].append(i)

Prediction function for clusters:

In [17]:
def predict_clusters(u, _clusters, _cl_array,smat,rmat,mc_mat, j):
    x = _clusters[u]#cluster number of required user
    num=0
    den=0
    s=0
    cnt=0
    for i in range(len(_cl_array[x])):
        s=s+mc_mat[_cl_array[x][i]][j]
        if mc_mat[_cl_array[x][i]][j]!=0:
            cnt = cnt+1
    if cnt!=0:
        s=s/cnt
    else:
        s=0
    for i in range(len(_cl_array[x])):
        if smat[_cl_array[x][i]][u]>0:
            num = num+smat[_cl_array[x][i]][u]*mc_mat[_cl_array[x][i]][j]
            den = den+smat[_cl_array[x][i]][u]
    if den==0:
        return 0
    else:
        return s+num/den

In [18]:
mean_centered=[[0 for i in range(cols)] for j in range(rows)]
for i in range(rows):
    s=0
    cnt=0
    for j in range(cols):
        s = s+rating_mat[i][j]
        if rating_mat[i][j]!=0:
            cnt=cnt+1
    if cnt==0:
        s=0
    else:
        s=s/cnt
    for j in range(cols):
        if rating_mat[i][j]!=0:
            mean_centered[i][j]=rating_mat[i][j]-s

Function for finding the mean absolute error:

In [20]:
def test_pred(tmat,smat):
    s=0
    cnt=0
    for i in range(rows):
        for j in range(cols):
            if tmat[i][j]!=0:
                x = predict_clusters(i,clusters,cl_array,sim_u,test_mat,mean_centered,j)
                y = tmat[i][j]
                s = s+abs(x-y)
                cnt = cnt+1
#                 if cnt%100==0:
#                     print("i: "+str(i)+" j: "+str(j)+" x: "+str(x)+" y: "+str(y))
    return (s/cnt)

In [21]:
print(test_pred(test_mat,sim_u))

3.456263925200501


For finding the rms error:

In [22]:
import math
s=0
cnt=0
for i in range(rows):
    for j in range(cols):
        if test_mat[i][j]!=0:
            x = predict_clusters(i,clusters,cl_array,sim_u,rating_mat,mean_centered, j)
            y = test_mat[i][j]
            s = s+(x-y)*(x-y)
            cnt = cnt+1
#                 if cnt%100==0:
#                     print("i: "+str(i)+" j: "+str(j)+" x: "+str(x)+" y: "+str(y))
if cnt==0:
    print(-1)
else:
    print(math.sqrt(s/cnt))

3.6133511838288386


# Data Smoothing and Clustering

Finding the smoothened matrix as complete matrix:

In [4]:
complete_matrix=[[0 for i in range(cols)] for j in range(rows)]

In [ ]:
# data smoothing->don't execute, can read from file
for i in range(rows):
    s=0
    cnt=0
    for j in range(cols):
        if rating_mat[i][j]!=0:
            cnt=cnt+1
            s = s+rating_mat[i][j]
    if cnt==0:
        s=0
    else:
        s=s/cnt
    for j in range(cols):
        if i%1000==0 and j%1000==0:
            print("i: "+str(i)+" j: "+str(j))
        x = clusters[i]
        dev=0
        for u in range(len(cl_array[x])):
            if rating_mat[cl_array[x][u]][j]!=0:
                dev = dev+abs(rating_mat[cl_array[x][u]][j]-s)
        if cnt==0:
            complete_matrix[i][j]=0
        else:
            dev=dev/cnt
            complete_matrix[i][j]=s+dev

In [12]:
f=open('./complete_matrix.csv','w+')
for i in range(rows):
    for j in range(cols):
        f.write(str(i)+','+str(j)+','+str(complete_matrix[i][j])+'\n')
f.close()

In [5]:
with open('./complete_matrix.csv','rt')as f:
    data = csv.reader(f)
    i=0
    for row in data:
        complete_matrix[int(row[0])][int(row[1])]=float(row[2])

In [13]:
import math
def similarity(u1,u2,rmat,cmat,w):#lambda = w if user has rated an item, else 1-w
    num=0
    s1=0
    s2=0
    for i in range(cols):
        s1=s1+cmat[u1][i]
        s2=s2+cmat[u2][i]
    s1=s1/cols
    s2=s2/cols
    den1=0
    den2=0
    for i in range(cols):
        if rmat[u1][i]!=0:
            num=num+w*(cmat[u1][i]-s1)*(cmat[u2][i]-s2)
        else:
            num=num+(1-w)*(cmat[u1][i]-s1)*(cmat[u2][i]-s2)
        if rmat[u1][i]!=0:
            den1=den1+w*(cmat[u1][i]*cmat[u1][i])
        else:
            den1=den1+(1-w)*(cmat[u1][i]*cmat[u1][i])
        den2=den2+(cmat[u2][i]*cmat[u2][i])
    den1=math.sqrt(den1)
    den2=math.sqrt(den2)
    return num/(den1*den2)

In [14]:
sim_clusters=[[0 for i in range(rows)] for j in range(rows)]

In [ ]:
#don't execute
# f = open("sim_clusters_2.csv","w+")
# for i in range(rows):
#     j=i+1
#     while j!=rows:
#         sim_clusters[i][j]=similarity(i,j,rating_mat,complete_matrix,0.8)
#         if j%100==0:
#             print("i: "+str(i)+" j: "+str(j)+" "+str(sim_clusters[i][j]))
#         f.write(str(i)+", "+str(j)+", "+str(sim_clusters[i][j])+"\n")
#         j=j+1
# f.close()

In [15]:
with open('./sim_clusters_2.csv','rt')as f:
    data = csv.reader(f)
    i=0
    for row in data:
        sim_clusters[int(row[0])][int(row[1])]=float(row[2])

Prediction function for smoothened matrix:

In [28]:
def predict_clusters2(u,j,_sim_u,rmat,w):
    rua=0
    for l in range(cols):
        rua=rua+rmat[u][l]
    rua=rua/cols
    num=0
    den=0
    x = clusters[u]
    lis = cl_array[x]
    s=0
    cnt=0
    for i in range(len(lis)):
        s=s+rmat[lis[i]][j]
        if rmat[i][j]!=0:
            cnt=cnt+1
        if cnt==0:
            s=0
        else:
            s=s/cnt
    for i in range(len(lis)):
        num = num+w*sim_u[lis[i]][u]*(rmat[i][j]-s)
        den=den+w*sim_u[lis[i]][u]
    return rua+(num/den)

In [29]:
print(predict_clusters2(1,2,sim_u,rating_mat,0.8))

1.4473969151585104


In [19]:
with open('./sim_clusters_2.csv','rt')as f:
    data = csv.reader(f)
    i=0
    for row in data:
        sim_clusters[int(row[0])][int(row[1])]=float(row[2])
        sim_clusters[int(row[1])][int(row[0])]=float(row[2])   

Finding the mean absolute error for different values of hyperparameter w:

In [ ]:
s=0
cnt=0
for i in range(rows):
    for j in range(cols):
        if test_mat[i][j]!=0:
            x = predict_clusters2(i,j,sim_u,rating_mat,555555)
            y = test_mat[i][j]
            if j%500==0:
                print("i: "+str(i)+" j: "+str(j)+" "+str(s))
            s = s+abs(x-y)
            cnt = cnt+1
print(s/cnt)
# return (s/cnt) 9.140008516577218

Finding the rms error for different values of hyperparameter w:

In [ ]:
s=0
cnt=0
for i in range(rows):
    for j in range(cols):
        if test_mat[i][j]!=0:
            x = predict_clusters2(i,j,sim_clusters,rating_mat,20,0.8)
            y = test_mat[i][j]
            s = s+(x-y)*(x-y)
            cnt = cnt+1
#                 if cnt%100==0:
#                     print("i: "+str(i)+" j: "+str(j)+" x: "+str(x)+" y: "+str(y))
if cnt==0:
    print(-1)
else:
    print(math.sqrt(s/cnt))